In [ ]:
'''
Name: model-development.(ipynb/py)
Description: Notebook for continuous model development, tweaking, and testing
Authors: Landen Doty, Sepehr Noori
Date: 10/03/23
'''

# Import Packages

In [ ]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


# Import Data from Drive

In [ ]:
import os
import zipfile
# mounting local google drive
from google.colab import drive
drive.mount('/content/drive')

# copying local drive data to colab
!mkdir /tmp/data

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
mkdir: cannot create directory ‘/tmp/data’: File exists


# Binary Cars

In [ ]:
# Unzip and store data on colab instance
!cp "/content/drive/MyDrive/Colab Notebooks/ParkSense/vehicle_detection_data.zip" "/tmp/data/"
zip_file = '/tmp/data/vehicle_detection_data.zip'
zip_ref = zipfile.ZipFile(zip_file, 'r')
zip_ref.extractall('/tmp/data/vehicle_data')
zip_ref.close()


In [ ]:
# path to datasets
vehicle = '/tmp/data/vehicle_data/data/vehicles/'
non_vehicle = '/tmp/data/vehicle_data/data/non-vehicles/'


In [ ]:
# upload our setup script
from google.colab import files
uploaded = files.upload()

Saving setup-data.sh to setup-data (1).sh


# Split Dataset into training and validation

In [ ]:
! bash setup-data.sh

In [ ]:
! rm -r /tmp/data/vehicle_data/data/vehicles
! rm -r /tmp/data/vehicle_data/data/non-vehicles/

# Data Generators

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Create training and validation data generators
# Currently normalizes images, but commented lines can be used for image augmentation if our model requires re-training with more creative datasets

# validation split allows us to use 25% of the specified dir. for training
train_datagen = ImageDataGenerator(
      rescale=1./255,
      validation_split=.75
      #rotation_range=40,
      #width_shift_range=0.2,
      #height_shift_range=0.2,
      #shear_range=0.2,
      #zoom_range=0.2,
      #horizontal_flip=True,
      #fill_mode='nearest'
      )

# validation split allows us to use 25% of the specified dir. for validation
validation_datagen = ImageDataGenerator(
      rescale=1./255,
      validation_split=.25
      #rotation_range=40,
      #width_shift_range=0.2,
      #height_shift_range=0.2,
      #shear_range=0.2,
      #zoom_range=0.2,
      #horizontal_flip=True,
      #fill_mode='nearest'
      )

# generate images from our extracted data set
# VGG16 used 150x150x3 input image size

# training images
train_generator = train_datagen.flow_from_directory(
        '/tmp/data/vehicle_data/data/training/',
        target_size=(150, 150),
        batch_size=32,
        subset='training',
        class_mode='binary')

# validation images
validation_generator = validation_datagen.flow_from_directory(
        '/tmp/data/vehicle_data/data/test/',
        target_size=(150, 150),
        batch_size=32,
        subset='validation',
        class_mode='binary')

Found 2220 images belonging to 2 classes.
Found 2220 images belonging to 2 classes.


# Define and Train Model

In [ ]:
import tensorflow_hub as hub
import tensorflow_datasets as tfds

from tensorflow.keras.applications.vgg16 import VGG16

IMAGE_SHAPE = (150,150)

# We are using transfer learning for the convolutional layers of our model
# VGG16 is frequently used for car model classification/identification
# Set trainable to false so we don't retrain those layers
base_model = VGG16(weights="imagenet", include_top=False, input_shape=IMAGE_SHAPE+(3,))
base_model.trainable = False

# Build the entire model
# Additional hidden layer with 10 neurons
# Output layer has a single neuron for binary classification
model = tf.keras.Sequential([
        base_model,
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(10, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
])

# Used for our testing and iteration => total params and size
model.summary()

model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001),
              metrics=['acc'])

# 3 epochs yield ~ 99% accuracy
model.fit(
      train_generator,
      epochs=3,
      validation_data=validation_generator)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten_2 (Flatten)         (None, 8192)              0         
                                                                 
 dense_4 (Dense)             (None, 10)                81930     
                                                                 
 dense_5 (Dense)             (None, 1)                 11        
                                                                 
Total params: 14796629 (56.44 MB)
Trainable params: 81941 (320.08 KB)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________
Epoch 1/3
70/70 [==============================] - 1254s 18s/step - loss: 0.2923 - acc: 0.9532 - val_loss: 0.1617 - val_acc: 0.9869
Epoch 2/